In [14]:
BACKEND_URLS="http://10.1.7.221:8000,http://10.1.7.221:8001,http://10.1.7.221:8002"
CLOUD_SERVER_URL="http://10.1.7.221:8003"
CPU_THRESHOLD=5000
MEM_THRESHOLD=5000
CHECK_INTERVAL=30

In [ ]:
import numpy as np
import tensorflow as tf
import os
import requests
from tensorflow.keras.losses import mse

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
BACKEND_URLS = BACKEND_URLS.split(",")
CLOUD_SERVER_URL = CLOUD_SERVER_URL
MEM_THRESHOLD = float(MEM_THRESHOLD)
CPU_THRESHOLD = float(CPU_THRESHOLD)
CHECK_INTERVAL = float(CHECK_INTERVAL)

# Load LSTM model
model = tf.keras.models.load_model('lstm_model.h5', compile=False)

# Global variable to track best backend safely
best_backend_url = None

def periodic_health_check():
    global best_backend_url

    best_node = None
    best_score = float('inf')  # lower combined CPU+MEM is better

    for backend_url in BACKEND_URLS:
        try:
            metrics_response = requests.get(f"{backend_url}/metrics", timeout=5)
            if metrics_response.status_code == 200:
                data = metrics_response.json()
                instances_raw = data.get("instances")
                # Convert list of dicts into (10, 6) array
                print(instances_raw)
                last_10_instances = np.array([
                    [
                        inst["current_requests"],
                        inst["cpu_percent"],
                        inst["memory_percent"],
                        inst["net_bytes_sent"],
                        inst["net_bytes_recv"],
                        inst["num_threads"]
                    ]
                    for inst in instances_raw
                ])
                last_10_instances = scaler.fit_transform(last_10_instances) 
                print(last_10_instances)
                if last_10_instances.shape == (10, 6):
                    input_data = last_10_instances.reshape((1, 10, 6))
                    print(input_data)
                    prediction = model.predict(input_data, verbose=0)
                    predicted_cpu = prediction[0][0]
                    predicted_mem = prediction[0][1]
                    combined_score = predicted_cpu + predicted_mem

                    print(f"[{backend_url}] CPU: {predicted_cpu:.4f}, MEM: {predicted_mem:.4f}, Score: {combined_score:.4f}")

                    if (predicted_cpu < CPU_THRESHOLD and
                        predicted_mem < MEM_THRESHOLD and
                        combined_score < best_score):
                        best_node = backend_url
                        best_score = combined_score
                else:
                    print(f"[{backend_url}] Invalid shape after extraction: {last_10_instances.shape}")
            else:
                print(f"[{backend_url}] Failed to get metrics (status {metrics_response.status_code})")
        except Exception as e:
            print(f"[{backend_url}] Error: {e}")
        break
        

    best_backend_url = best_node

    if best_node:
        print(f"[Health Check] Best backend selected: {best_node} (Score: {best_score:.4f})")
    else:
        print("[Health Check] No backend under threshold; will use cloud fallback")
periodic_health_check()


[{'current_requests': 0, 'cpu_percent': 4.9, 'memory_percent': 47.5, 'net_bytes_sent': 54907, 'net_bytes_recv': 24849, 'num_threads': 27}, {'current_requests': 0, 'cpu_percent': 0.0, 'memory_percent': 47.5, 'net_bytes_sent': 54907, 'net_bytes_recv': 24849, 'num_threads': 27}, {'current_requests': 0, 'cpu_percent': 0.0, 'memory_percent': 47.6, 'net_bytes_sent': 54907, 'net_bytes_recv': 24849, 'num_threads': 27}, {'current_requests': 0, 'cpu_percent': 0.0, 'memory_percent': 47.6, 'net_bytes_sent': 54907, 'net_bytes_recv': 24849, 'num_threads': 27}, {'current_requests': 0, 'cpu_percent': 1.2, 'memory_percent': 47.6, 'net_bytes_sent': 54907, 'net_bytes_recv': 24849, 'num_threads': 27}, {'current_requests': 0, 'cpu_percent': 0.0, 'memory_percent': 47.6, 'net_bytes_sent': 54907, 'net_bytes_recv': 24849, 'num_threads': 27}, {'current_requests': 0, 'cpu_percent': 1.2, 'memory_percent': 47.6, 'net_bytes_sent': 54907, 'net_bytes_recv': 24849, 'num_threads': 27}, {'current_requests': 0, 'cpu_perc

In [ ]:
print(type(BACKEND_URLS))

In [1]:
import tensorflow as tf
print(tf.__version__)


2.18.0
